In [8]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from tqdm.notebook import tqdm
import warnings
warnings. filterwarnings('ignore')

In [2]:
! pip install Sastrawi
! pip install tqdm

In [3]:
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

True

In [4]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [5]:
df = pd.read_csv(r'D:\SKRIPSI\skripsi_2025\dataset berita terkini 3\politik_merge_compressed.csv.gz', compression='gzip')
df

,Judul,Waktu,Link,Content,source
0,Jokowi Kenakan Pakaian Adat Betawi di Sidang T...,16/08/2024,https://nasional.kompas.com/read/2024/08/16/11...,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...",kompas
1,Amnesty International Beberkan 6 Indikator Kri...,7/18/2024,https://nasional.tempo.co/read/1893144/amnesty...,"TEMPO.CO, Jakarta - Amnesty International Indo...",tempo
2,"Jelang Long Weekend, Stasiun Kereta Cepat Hali...","Rabu, 08 Mei 2024 19:18 WIB",https://news.detik.com/berita/d-7331666/jelang...,"Stasiun kereta cepat Whoosh di Halim, Jakarta ...",detik
3,KPU Tegaskan Pemilih Tak Terdaftar di DPT Bisa...,13/02/2024,https://nasional.kompas.com/read/2024/02/13/21...,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (...",kompas
4,Kemenag Luncurkan Gerakan Senam Haji Jaga Keta...,4/29/2024,https://nasional.tempo.co/read/1861810/kemenag...,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem...",tempo
...,...,...,...,...,...
35864,Kapolri Tegaskan Contraflow di Tol Tetap Dibut...,"Senin, 08 Apr 2024 20:03 WIB",https://news.detik.com/berita/d-7285216/kapolr...,Kapolri Jenderal Listyo Sigit Prabowo menegask...,detik
35865,Temuan Cairan Misterius di Rumah Eks Bupati Je...,"Sabtu, 10 Agu 2024 16:16 WIB",https://news.detik.com/berita/d-7483244/temuan...,Polisi melakukan olah tempat kejadian perkara ...,detik
35866,Danjen Kopassus Bukber dengan Para Mantan Danj...,"Jumat, 05 Apr 2024 20:53 WIB",https://news.detik.com/berita/d-7281174/danjen...,Danjen Kopassus Mayjen TNI Djon Afriandi mengg...,detik
35867,Pertemuan Jokowi dan Paus Fransiskus Akan Baha...,3/9/2024,https://nasional.kompas.com/read/2024/09/03/14...,"TANGERANG, KOMPAS.com - Menteri Agama Yaqut C...",kompas


In [6]:
df = df[['Content', 'Judul']]

In [10]:
df = df.drop_duplicates(subset=['Content'])
df = df.dropna(subset=['Content', 'Judul'])
print(f"Dataset memiliki {len(df)} baris")

Dataset memiliki 35566 baris


In [11]:
sample_df = df.sample(n=1000, random_state=42)
print(f"Sample berhasil diambil dengan {len(sample_df)} baris")

Sample berhasil diambil dengan 1000 baris


PREPROCESSING DATA

In [14]:
#Cleaning Data

def clean_berita(text):
    # Penanganan nilai null
    if not isinstance(text, str):
        text = str(text)

    text = text.replace('â€œ', '').replace('â€', '')
    text = re.sub(r'^[A-Z]+, [A-Za-z0-9\.]+(\.com)? - ', '', text)
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'[^A-Za-z0-9.,;:!?\'" ]', ' ', text)

    # Normalisasi whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [15]:
# Normalisasi
#Normalisasi
norm_berita = {
    # Singkatan media dan lokasi
    " kompas.com ": " ",
    " tempo.co ": " ",
    " detikcom ": " ",

    # Singkatan waktu dan ukuran
    " wib ": " waktu indonesia barat ",
    " km ": " kilometer ",
    " m ": " meter ",

    # Singkatan jabatan dan institusi
    " presiden ": " presiden ",
    " ri ": " republik indonesia ",
    " dpr ": " dewan perwakilan rakyat ",
    " dpd ": " dewan perwakilan daerah ",
    " mpr ": " majelis permusyawaratan rakyat ",
    " pemilu ": " pemilihan umum ",
    " kpu ": " komisi pemilihan umum ",
    " ksp ": " kantor staf presidenan ",
    " ikn ": " ibu kota nusantara ",
    " tni ": " tentara nasional indonesia ",
    " polri ": " kepolisian republik indonesia ",
    " kpk ": " komisi pemberantasan korupsi ",
    " pdip ": " partai demokrasi indonesia perjuangan ",
    " lsi ": " lembaga survei indonesia ",
    " jppr ": " jaringan pendidikan pemilih untuk rakyat ",
    " uu ": " undang-undang ",
    " ruu ": " rancangan undang-undang ",
    " ham ": " hak asasi manusia ",
    " ite ": " informasi dan transaksi elektronik ",

    # Nama-nama yang sering muncul
    " jokowi ": " joko widodo ",
    " puan maharani ": " puan maharani ",
    " megawati soekarnoputri ": " megawati soekarnoputri ",
    " prabowo ": " prabowo ",

    # Singkatan umum dalam berita
    " yg ": " yang ",
    " dg ": " dengan ",
    " dlm ": " dalam ",
    " tsb ": " tersebut ",
    " tgl ": " tanggal ",
    " jk ": " jika ",
    " pd ": " pada ",
    " dr ": " dari ",
    " utk ": " untuk ",
    " spy ": " supaya ",
    " thn ": " tahun ",
    " sbb ": " sebagai berikut ",
    " jt ": " juta ",
    " tdk ": " tidak ",
    " sdh ": " sudah ",
    " blm ": " belum ",
    " hrs ": " harus ",
    " jl ": " jalan ",
    " no ": " nomor ",
    " bpk ": " bapak ",
    " sdr ": " saudara ",

    # Kata serapan yang sering salah tulis dan simbol
    " tehnologi ": " teknologi ",
    " aktifitas ": " aktivitas ",
    " sistim ": " sistem ",
    " intruksi ": " instruksi ",
    " trima ": " terima ",
    " whoosh ": " whoosh ",
    " -Â ": " - ",
    " Â ": " ",

    # Kata-kata informal
    " gak ": " tidak ",
    " nggak ": " tidak ",
    " enggak ": " tidak ",
    " udah ": " sudah ",
    " aja ": " saja ",
    " doang ": " saja ",
    " gede ": " besar ",
    " banget ": " sangat ",
}

def normalisasi_berita(text):
    # Memastikan input adalah string
    if not isinstance(text, str):
        text = str(text)

    # Tambahkan spasi di awal dan akhir untuk memudahkan penggantian
    text = " " + text + " "

    # Lakukan penggantian berdasarkan kamus normalisasi
    for key, value in norm_berita.items():
        text = text.replace(key, value)

    # Hapus spasi berlebih dan kembalikan
    return re.sub(r'\s+', ' ', text).strip()

In [16]:
# Case Folding
def case_folding_berita(text):
    if not isinstance(text, str):
        text = str(text)
    return text.lower()

In [17]:
# Tokenizing
def tokenizing(text):
    return word_tokenize(text)

In [21]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Stopwords gabungan dari NLTK dan kustom
stop_words = set(stopwords.words('indonesian'))
custom_stopwords = {
    'ya', 'nya', 'juga', 'ini', 'itu', 'dan', 'di', 'ke', 'dengan',
    'untuk', 'pada', 'ada', 'yang', 'dari', 'oleh', 'tersebut',
    'bisa', 'dapat', 'seperti', 'karena', 'saat'
}
stop_words.update(custom_stopwords)

In [22]:
def process_all_steps(text):
    # Penanganan null
    if not isinstance(text, str):
        text = str(text)
        
    # Cleaning
    text = clean_berita(text)
    
    # Normalisasi
    text = normalisasi_berita(text)
    
    # Case folding
    text = case_folding_berita(text)
    
    # Tokenizing
    tokens = word_tokenize(text)
    
    # Filtering stopwords
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # Stemming
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    
    # Join kembali menjadi teks
    return ' '.join(stemmed_tokens)

In [23]:
print("\nMemproses kolom Content...")
tqdm.pandas(desc="Processing Content")
sample_df['processed_content'] = sample_df['Content'].progress_apply(process_all_steps)

print("\nMemproses kolom Judul...")
tqdm.pandas(desc="Processing Judul")
sample_df['processed_judul'] = sample_df['Judul'].progress_apply(process_all_steps)


Memproses kolom Content...


Processing Content:   0%|          | 0/1000 [00:00<?, ?it/s]


Memproses kolom Judul...


Processing Judul:   0%|          | 0/1000 [00:00<?, ?it/s]

In [24]:
output_file = 'processed_sample_1000.csv'
sample_df.to_csv(output_file, index=False)
print(f"\nProses selesai! Hasil disimpan ke {output_file}")


Proses selesai! Hasil disimpan ke processed_sample_1000.csv


In [29]:
from IPython.display import display
display(sample_df[['Content', 'processed_content', 'Judul', 'processed_judul']].head())

,Content,processed_content,Judul,processed_judul
11082,"TEMPO.CO, Jakarta - Sejak 26 tahun yang lalu, ...",tempo co jakarta 26 mei kenang lahir reforma...,Bara Reformasi Terus Dihidupkan: Aksi Kamisan ...,bara reformasi hidup aksi kamis adil korban culi
21104,"TEMPO.CO, Jakarta - Menteri Koordinator Pereko...",tempo co jakarta menteri koordinator ekonomi ...,Airlangga soal Ratas Prabowo dengan Jokowi: Ba...,airlangga ratas prabowo jokowi bahas perintah
32932,"TEMPO.CO, Yogyakarta - Capres nomor urut 03 Ga...",tempo co yogyakarta capres nomor urut 03 ganj...,"Prabowo-Gibran Gencar Gerak di Jateng, Ganjar ...",prabowo gibran gencar gerak jateng ganjar kan...
21779,KOMPAS.com – Partai Keadilan Sejahtera (PKS) ...,kompas com partai adil sejahtera pks buka opsi...,Soal Sinyal PKS Tinggalkan Anies dan Gabung KI...,sinyal pks tinggal anies gabung kim plus zulh...
32515,Dalam sejarahnya selama menjadi Inspektur Upac...,sejarah inspektur upacara hut merdeka ri pres...,Daftar Baju Adat yang Pernah Dipakai Jokowi sa...,daftar baju adat pakai jokowi upacara 17 agustus


LABELING

In [11]:
def label_sentimen_ekonomi(processed_text):
    # Definisikan kategori sentimen untuk konteks ekonomi/nilai tukar
    sentiment_categories = {
        'penguatan_rupiah': [
            'menguat', 'naik', 'apresiasi', 'positif', 'surplus', 'berkembang',
            'tumbuh', 'stabil', 'kokoh', 'fundamental', 'kepercayaan', 'investasi'
        ],
        'pelemahan_rupiah': [
            'melemah', 'turun', 'depresiasi', 'negatif', 'defisit', 'penurunan', 
            'krisis', 'resesi', 'inflasi', 'tekanan', 'ketidakpastian', 'volatil'
        ],
        # Faktor-faktor ekonomi dari proposal
        'faktor_penguatan': [
            'ekspor', 'cadangan', 'devisa', 'investasi', 'pertumbuhan', 'ekonomi',
            'kebijakan', 'moneter', 'stabilitas', 'politik', 'kepercayaan', 'investor'
        ],
        'faktor_pelemahan': [
            'inflasi', 'impor', 'utang', 'defisit', 'anggaran', 'ketidakpastian',
            'politik', 'global', 'minyak', 'krisis', 'konflik', 'perang'
        ]
    }
    
    # Pisahkan kata-kata teks
    words = processed_text.split()
    
    # Hitung skor untuk setiap kategori
    scores = {category: 0 for category in sentiment_categories}
    
    for word in words:
        for category, category_words in sentiment_categories.items():
            if word in category_words:
                scores[category] += 1
    
    # Tentukan sentimen berdasarkan skor kategori
    penguatan_score = scores['penguatan_rupiah'] + scores['faktor_penguatan']
    pelemahan_score = scores['pelemahan_rupiah'] + scores['faktor_pelemahan']
    
    # Klasifikasi biner (hanya positif atau negatif)
    if penguatan_score >= pelemahan_score:
        return 'positif'  # Berita berpotensi memperkuat rupiah
    else:
        return 'negatif'  # Berita berpotensi melemahkan rupiah

In [12]:
def label_sentimen_lexicon(processed_text):
    # Definisikan kamus kata positif dan negatif
    positive_words = [
        'tumbuh', 'naik', 'positif', 'apresiasi', 'kuat', 'untung', 'stabil',
        'baik', 'optimis', 'tingkat', 'sukses', 'unggul', 'dorong', 'dukung',
        'laju', 'surplus', 'kokoh', 'angkat', 'dongkrak', 'pemulih', 'tekan',
        'perkuat', 'rekor', 'prestasi', 'potensial', 'prospek', 'efisien'
    ]
    
    negative_words = [
        'turun', 'lemah', 'negatif', 'rugi', 'krisis', 'defisit', 'buruk',
        'tekanan', 'resesi', 'inflasi', 'rendah', 'lambat', 'kendala', 'akibat',
        'masalah', 'sulit', 'khawatir', 'menurun', 'jatuh', 'tekor', 'tertekan',
        'beban', 'tergerus', 'risiko', 'ancam', 'hambat', 'kesulitan', 'terhambat'
    ]
    
    # Hitung jumlah kata positif dan negatif dalam teks
    words = processed_text.split()
    
    positive_count = sum(1 for word in words if word in positive_words)
    negative_count = sum(1 for word in words if word in negative_words)
    
    # Klasifikasi biner (hanya positif atau negatif)
    if positive_count >= negative_count:
        return 'positif'
    else:
        return 'negatif'

In [13]:
def combine_methods_binary(sample_df):
    # Terapkan kedua metode pelabelan pada konten berita
    sample_df['label_lexicon'] = sample_df['processed_content'].apply(label_sentimen_lexicon)
    sample_df['label_ekonomi'] = sample_df['processed_content'].apply(label_sentimen_ekonomi)
    
    # Gabungkan hasil dengan voting sederhana
    def vote_labels(row):
        labels = [row['label_lexicon'], row['label_ekonomi']]
        
        if labels.count('positif') > labels.count('negatif'):
            return 'positif'
        elif labels.count('negatif') > labels.count('positif'):
            return 'negatif'
        else:
            # Jika terjadi keseimbangan, berikan prioritas pada metode ekonomi
            # karena lebih sesuai dengan konteks penelitian Anda
            return row['label_ekonomi']
    
    sample_df['final_label'] = sample_df.apply(vote_labels, axis=1)
    return sample_df

In [14]:
sample_df = pd.read_csv('processed_sample_1000.csv')
labeled_df = combine_methods_binary(sample_df)
print("Distribusi label sentimen:")
print(labeled_df['final_label'].value_counts())
labeled_df.to_csv('labeled_dataset.csv', index=False)

Distribusi label sentimen:
positif    859
negatif    141
Name: final_label, dtype: int64
